### Preparing the Environment

In [1]:
from openai import OpenAI

In [25]:
with open("keys.txt",'r') as f:
    key = f.read().strip()

In [5]:
client = OpenAI(
    api_key=key,  # the keys from plateform.openai.com, this needs to be payed
    base_url="https://api.deepseek.com"
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    
    ],
    stream= False
)

In [6]:
print(response.choices[0].message.content)



The capital of France is **Paris**. It is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.  

Would you like any additional information about Paris or France?


### Retrieval

In [16]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json','r') as f_in:
    docs_raw = json.load(f_in)


In [6]:
documents=[]
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
documents = [
    {**doc, 'course': course_dict['course']}
    for course_dict in docs_raw
    for doc in course_dict['documents']
]
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'], 
    keyword_fields=['course'],
)

In [22]:
index.fit(documents)
q = 'Course - What are the prerequisites for this course?'
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict =boost,
    num_results=10)
results

[{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython

In [7]:
# help(index.search)


### Generation with OpenAI

In [14]:
from openai import OpenAI

In [9]:
q = 'Course - What are the prerequisites for this course?'


In [11]:
client = OpenAI(
    api_key=key,  # the keys from plateform.openai.com, this needs to be payed
    base_url="https://api.deepseek.com"
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages = [{"role":"user","content":q}])

print(response.choices[0].message.content)

The prerequisites for a course can vary depending on the subject and level of the course. Could you please specify:  

1. **The course name or subject area** (e.g., *Introduction to Machine Learning, Calculus II, Digital Marketing*)?  
2. **The institution or platform** offering it (e.g., *Coursera, MIT, Udemy*)?  

For example:  
- *"What are the prerequisites for Harvard's CS50?"*  
- *"Are there prerequisites for the Google Data Analytics Certificate on Coursera?"*  

Let me know, and I’ll provide a precise answer!


In [25]:
prompt_template ="""
You are a course assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
USE only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, say "I don't know".

QUESTION: {question}
CONTEXT: {context}

""".strip()


In [30]:
context=""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [32]:
prompt = prompt_template.format(question=q, context=context).strip()

print(prompt)

You are a course assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
USE only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, say "I don't know".

QUESTION: Course - What are the prerequisites for this course?
CONTEXT: section: General course-related questions
question: Course - What are the prerequisites for this course?
answer: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's 

In [ ]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages = [{"role":"user","content":prompt}])

print(response.choices[0].message.content)

### Cleaned RAG flow

In [33]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'], 
    keyword_fields=['course'],
)
index.fit(documents)

def search(query):
    """this function using the backage minsearch to get the data from the documents.json file"""
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10)
    return results

In [15]:
def build_prompt(query, search_results):
    """this function builds the prompt for the model"""
    prompt_template = """
    You are a course assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    USE only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT does not contain the answer, say "I don't know".

    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return prompt_template.format(question=query, context=context).strip()

In [26]:
with open("keys.txt",'r') as f:
    key = f.read().strip()
    
client = OpenAI(
    api_key=key,  # the keys from plateform.openai.com, this needs to be payed
    base_url="https://api.deepseek.com"
)

def llm(prompt):
    """this function calls the model with the prompt"""

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
def rag(query):
    """this function combines the search and the llm"""
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
query = "How do i run kafka?"
answer = rag(query)

### Search with Elasticsearch

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from elasticsearch import Elasticsearch

In [10]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '331004c32b20', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'peFoFunQQv68qyiedN8gHw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
# create a index in elasticsearch
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [18]:
def es_search(query):
    """this function uses elasticsearch to search for the query"""
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    return result_docs

In [22]:
def rag(query):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
query = 'I just disovered the course. Can I still join it?'

In [33]:
es_search(query)


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [27]:
rag(query)


"Yes, you can still join the course even after the start date. You are eligible to submit the homeworks, but be mindful of the deadlines for the final projects. Additionally, all course materials will remain available after the course finishes, allowing you to follow the course at your own pace. \n\nYou can also get support in the self-paced mode through the Slack channel, where you can ask questions or search for existing answers. \n\nIf you haven't registered, that's fine too—registration is only to gauge interest, and you can start learning and submitting homework without it."

#### homework

In [38]:
query = """"How do execute a command on a Kubernetes pod?"""
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
results = es_client.search(index=index_name, body=search_query)


In [41]:
top_hit = results["hits"]["hits"][0]
score = top_hit["_score"]
print(score)

31.973522


In [ ]:
query = """"How do execute a command on a Kubernetes pod?"""
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
results = es_client.search(index=index_name, body=search_query)


ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 303, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'GM8sP5cB42VBJi6I_Ade', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'p88tP5cB42VBJi6IGAew', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kuberne

In [43]:
# get the 3th hit
hits = results["hits"]["hits"]

if len(hits) >= 3:
    third_question  = hits[2]["_source"]["question"]
    print(third_question)

How do I copy files from a different folder into docker container’s working directory?


In [58]:
query = """"How do execute a command on a Kubernetes pod?"""
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
results = es_client.search(index=index_name, body=search_query)
third_reult = results["hits"]["hits"][2]["_source"]



prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


context=""

context += f"section: {third_reult['section']}\nquestion: {third_reult['question']}\nanswer: {third_reult['text']}\n\n"
prompt = prompt_template.format(question=query, context=context).strip()

In [60]:
len(prompt)

712

In [62]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: "How do execute a command on a Kubernetes pod?

CONTEXT:
section: 5. Deploying Machine Learning Models
question: How do I copy files from a different folder into docker container’s working directory?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:
COPY ["src/predict.py", "models/xgb_model.bin", "./"]											Gopakumar Gopinathan


In [64]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
num_tokens = len(tokens)

In [65]:
print(num_tokens)

153
